In [10]:
# Recurrent Neural Networks - Obama vs Trump: RNN model to classify between Obama/Trump tweets
# Shomik Jain, USC CAIS++

In [11]:
# Recurrent Neural Networks (RNNs)
# designed to work on temporal/sequential data
# RNN neurons have "memory" which caputres info about what has been calculated so far 
# limitations: cannot handle long term dependencies, suffer from vanishing gradients

In [12]:
# Long Short Term Memory Network: LSTM
# manage cell state with a variety of gates to more effectively control flow of info through time

In [13]:
# 1. Load in the Data

TWEETS_DIR = './obama-trump-data/tweets'
LABELS_DIR = './obama-trump-data/labels_np'
EMBEDDINGS_DIR = './word_vectors.txt'

In [14]:
import pickle

# Load tweets, labels
tweets = pickle.load(open(TWEETS_DIR,'rb'))
labels = pickle.load(open(LABELS_DIR,'rb'))

# Sample some tweets to display
for i in range(0,100,10):
    print("Tweet: ", tweets[i], ". Label: ", labels[i])

Tweet:  "America should be very proud." President Obama #LoveWins . Label:  0
Tweet:  TO ALL AMERICANS
https://t.co/D7Es6ie4fY . Label:  1
Tweet:  No deal was made last night on DACA. Massive border security would have to be agreed to in exchange for consent. Would be subject to vote. . Label:  1
Tweet:  I was viciously attacked by Mr. Khan at the Democratic Convention. Am I not allowed to respond? Hillary voted for the Iraq war, not me! . Label:  1
Tweet:  .@RudyGiuliani, one of the finest people I know and a former GREAT Mayor of N.Y.C., just took himself out of consideration for "State". . Label:  1
Tweet:  The #TPP establishes the highest labor standards of any trade agreement in history. http://t.co/3vqoancyYp . Label:  0
Tweet:  Thoughts and prayers to the great people of Indiana. You will prevail! . Label:  1
Tweet:  Hillary said she was under sniper fire (while surrounded by USSS.) Turned out to be a total lie. She is not fit to https://t.co/hBIrGj21l6 . Label:  1
Tweet:  The f

In [15]:
# 2. Data Preprocessing

# 2.1 -- Tokenize the tweets: convert each tweet into a sequence of word indices
# 2.2 -- Pad the input sequences with blank spots at end to make sure they're all same length
# 2.3 -- Load in pre-trained word embeddings to convert each word index=>unique word embedding vector

# Word embeddings are a way of encoding words into n-dimensional vectors with continuous values
# so that the vector contains some information about the words' meanings

In [16]:
import numpy as np
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenize the tweets (convert sentence to sequence of words)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tweets)

sequences = tokenizer.texts_to_sequences(tweets)
word_index = tokenizer.word_index

print('Found %s unique tokens (words).' % len(word_index))

# Pad sequences to ensure samples are the same size
training_data = pad_sequences(sequences)

print("Training data size is (%d,%d)"  % training_data.shape) # shape = (num. tweets, max. length of tweet)
print("Labels are size %d"  % labels.shape)

Found 13049 unique tokens (words).
Training data size is (6136,31)
Labels are size 6136


In [17]:
# Show effect of tokenization, padding
print("Original tweet: ", tweets[0])
print("Tweet after tokenization and padding: ", training_data[0])

Original tweet:  "America should be very proud." President Obama #LoveWins
Tweet after tokenization and padding:  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0  44  88  21  83 391  12  14 794]


In [18]:
# Convert words to word embedding vectors

EMBEDDING_DIM = 50
print('Indexing word vectors.')

import os
embeddings_index = {}
f = open(EMBEDDINGS_DIR)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# prepare word embedding matrix
num_words = len(word_index)+1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Indexing word vectors.
Found 400000 word vectors.


In [19]:
# Sample word embedding vector:
print(word_index["computer"]) # retrieve word index
print(embedding_matrix[2645]) # use word index to retrieve word embedding vector

2645
[ 0.079084   -0.81503999  1.79009998  0.91653001  0.10797    -0.55628002
 -0.84426999 -1.49510002  0.13417999  0.63626999  0.35146001  0.25813001
 -0.55028999  0.51055998  0.37408999  0.12092    -1.61660004  0.83653003
  0.14202    -0.52348     0.73452997  0.12207    -0.49079001  0.32532999
  0.45306    -1.58500004 -0.63848001 -1.00530005  0.10454    -0.42984
  3.18099999 -0.62186998  0.16819    -1.01390004  0.064058    0.57844001
 -0.45559999  0.73782998  0.37202999 -0.57722002  0.66441     0.055129
  0.037891    1.32749999  0.30991     0.50696999  1.23570001  0.1274
 -0.11434     0.20709001]


In [20]:
# 3. Create the classifier model

In [21]:
from keras.models import Sequential
from keras.layers import Embedding, Input
from keras.layers.merge import Concatenate
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Dropout, concatenate
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.optimizers import RMSprop
from keras import metrics
from keras.models import Model
import pickle

In [22]:
model = Sequential()

# Add pre-trained embedding layer 
# Converts word indices to GloVe word embedding vectors as they're fed in
model.add(Embedding(len(word_index) + 1,
                    EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=training_data.shape[1],
                    trainable=False))

# At this point, each individual training sample is now a sequence of word embedding vectors

model.add(LSTM(64, return_sequences = False)) #return_sequences: if multiple LSTM layers
model.add(Dropout(.2))

# Dense 1
model.add(Dense(32, activation='relu'))
model.add(Dropout(.2))

# Dense 2 (final vote)
model.add(Dense(1, activation = 'sigmoid'))

print(model.summary())

# Define Loss and Optimizer
LOSS = 'binary_crossentropy' #classifying between 0 and 1
OPTIMIZER = 'rmsprop' #RMSprop typically works better for RNNs per Keras

model.compile(loss = LOSS, optimizer = OPTIMIZER, metrics = [metrics.binary_accuracy])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 31, 50)            652500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 684,053
Trainable params: 31,553
Non-trainable params: 652,500
_________________________________________________________________

In [23]:
# 3. Train/Evaluate the Model

In [24]:
# number of epochs and batch size
EPOCHS = 10
BATCH_SIZE = 128

model.fit(training_data, labels, 
          epochs = EPOCHS, 
          batch_size = BATCH_SIZE, 
          validation_split =.2)

Train on 4908 samples, validate on 1228 samples
Epoch 1/10
4908/4908 [==============================] - 3s 622us/step - loss: 0.5055 - binary_accuracy: 0.7569 - val_loss: 0.3589 - val_binary_accuracy: 0.8485
Epoch 2/10
4908/4908 [==============================] - 2s 340us/step - loss: 0.3296 - binary_accuracy: 0.8659 - val_loss: 0.5451 - val_binary_accuracy: 0.7565
Epoch 3/10
4908/4908 [==============================] - 2s 344us/step - loss: 0.2855 - binary_accuracy: 0.8877 - val_loss: 0.2814 - val_binary_accuracy: 0.8738
Epoch 4/10
4908/4908 [==============================] - 2s 342us/step - loss: 0.2449 - binary_accuracy: 0.9026 - val_loss: 0.2505 - val_binary_accuracy: 0.8974
Epoch 5/10
4908/4908 [==============================] - 2s 351us/step - loss: 0.2185 - binary_accuracy: 0.9173 - val_loss: 0.2030 - val_binary_accuracy: 0.9243
Epoch 6/10
4908/4908 [==============================] - 2s 346us/step - loss: 0.2052 - binary_accuracy: 0.9201 - val_loss: 0.2049 - val_binary_accuracy: